In [ ]:
#import neccessary packages
import os
from sklearn import svm
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import array

#retrieve files
os.chdir('C:/Users/etxv15l/Desktop/Old Projects and Training/NIST Competition/comp_train')
train_directory = []
for file in os.listdir():
    train_directory.append(file)

os.chdir('C:/Users/etxv15l/Desktop/Old Projects and Training/NIST Competition/comp_test')
test_directory = []
for file in os.listdir():
    test_directory.append(file)

#get classes - train
train_classes = []
for i in range(len(train_directory)):
    if train_directory[i][0] == '1':
        train_classes.append(train_directory[i][:6])
    else:
        train_classes.append(train_directory[i][:5])

#get classes - test
test_classes = []
for i in range(len(test_directory)):
    if test_directory[i][0] == '1':
        test_classes.append(test_directory[i][:6])
    else:
        test_classes.append(test_directory[i][:5])      
        
#get img_arrays - train
os.chdir('C:/Users/etxv15l/Desktop/Old Projects and Training/NIST Competition/comp_train')
to_resize_train = []
for picture in train_directory:
    to_resize_train.append(cv2.imread(picture, 0))
    
#resize all images - train
to_flatten_train = []
for img in to_resize_train:
    to_flatten_train.append(cv2.resize(img, (128, 165)))
    
#flatten all resized images - train
flat_imgs_train = []
for img in to_flatten_train:
    flattened = img.flatten()
    flat_imgs_train.append(flattened)

#Xtrain for linear fitting
X_train = np.array(flat_imgs_train)

#get img_arrays - test
os.chdir('C:/Users/etxv15l/Desktop/Old Projects and Training/NIST Competition/comp_test')
to_resize_test = []
for picture in test_directory:
    to_resize_test.append(cv2.imread(picture, 0))
    
#resize all images - test
to_flatten_test = []
for img in to_resize_test:
    to_flatten_test.append(cv2.resize(img, (128, 165)))
    
#flatten all resized images - test
flat_imgs_test = []
for img in to_flatten_test:
    flattened = img.flatten()
    flat_imgs_test.append(flattened)

#X test for linear svm testing   
X_test = np.array(flat_imgs_test)

#ytrain and ytest
y_train = np.array(train_classes)
y_test = np.array(test_classes)

###Start Linear SVM model training/testing
#distinguish data and classes
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

#fit Linear SVM Model
clf = LinearSVC(random_state = 0)
clf.fit(X_train, y_train)

#get testing data predictions
preds = clf.predict(X_test)

#Put in dataframe form, print overall accuracy
e = {'img_name': test_directory, 'actual': test_classes, 'pred': preds}
full_results = pd.DataFrame(data = e)
full_results['Correct'] = np.where(full_results['actual'] == full_results['pred'], 1, 0)
print("Accuracy: " + str(sum(full_results['Correct'])/len(test_directory)))

#Get accuracy metrics
#Create confusion matrix and classification report - by form type
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
y_true = full_results['actual']
y_pred = full_results['pred']

labels = ['1040_1', '1040_2', 'sch_a', 'sch_b', 'sch_e']
cm = confusion_matrix(y_true, y_pred, labels)
cr = classification_report(y_true, y_pred)

print(cr)
print(cm)

#visualize results 
conf_arr = cm
norm_conf = []
for i in conf_arr:
    a = 0
    tmp_arr = []
    a = sum(i, 0)
    for j in i:
        tmp_arr.append(float(j)/float(a))
    norm_conf.append(tmp_arr)

fig = plt.figure()
plt.clf()
ax = fig.add_subplot(111)
ax.set_aspect(1)
res = ax.imshow(np.array(norm_conf), cmap = plt.cm.jet, interpolation = 'nearest')

width, height = conf_arr.shape
os.chdir('C:/Users/etxv15l/Desktop/') 
for x in range(width):
    for y in range(height):
        ax.annotate(str(conf_arr[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center')

cb = fig.colorbar(res)
labels = ['1040_1', '1040_2', 'sch_a', 'sch_b', 'sch_e']
plt.xticks(range(width), labels)
plt.yticks(range(height), labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png', format='png')